## About this notebook

I am experimenting with various graph and matrix libraries on the CPU and GPU.  For small graphs (less than 50k nodes) my theories are

* Dense operations will be faster
* Most/all sampling and graph distance metrics can be handled as matrix operations. 
  - this will cause signifcant speed up over implementations with loops
* GPU libraries, when you stick with matrix operations will be faster
  - this includes slicing
  
I'm unsure about the following

* GPU based operations are not finicky.  
* Libraries are easy to use. 
  - JAX is an easy-to-work-with replacement for numpy
  - cuGraph is an easy-to-work-with replacement for networkX

In [2]:
import trax
import numpy as np
import networkx as nx


In [3]:

#We'll just use trax to get to jax
trax.fastmath.use_backend('numpy')
from trax.fastmath import numpy as jnp


## Generate Test Graphs

Convert them to matrices for later

In [6]:
from networkx.generators.random_graphs import powerlaw_cluster_graph

In [7]:
G = powerlaw_cluster_graph(20000, 2, 0.2)

In [21]:
spl_gen = nx.all_pairs_shortest_path_length(G)

In [22]:
spl_dict = list(spl_gen)

In [20]:
len(spl_dict)

0

In [23]:
g_adj_mat = nx.adjacency_matrix(G)

can you calculate a "dense" graph matrix from G?

https://networkx.org/documentation/stable//reference/generated/networkx.linalg.graphmatrix.adjacency_matrix.html

## Calculate Derived Matrices

* Shortest Path
* Binning Matrix

In [4]:
from scipy.sparse.csgraph import shortest_path
from scipy.sparse import csr_matrix


In [36]:

def simple_bin_matrix(adjMat, oneHot=True):
    '''Construct a bin matrix for test purposes. 
    
    All nodes with the same degree end up in the same bin.
    
    Parameters
    ----------
    adjMat (ndarray): a Nodes x Nodes adjacency matrix (dense form)
    onHot (bool): should the results be returned 
    
    Returns
    ------
    ndarray: A Bins x Nodes 0,1 matrix if OneHot is True or a 1 x Nodes matrix where each entry is the degree of the node if oneHot is False.
    '''
    degrees      = np.sum(adjMat, axis=1)
    num_nodes, _ = adjMat.shape

    #TODO: refactor oneHot to a fuction. 
    if oneHot:
        out = np.zeros((num_nodes, np.max(degrees) + 1))
        out[np.arange(degrees.size), degrees] = 1
        return out.T
    else:
        return degrees



In [33]:
g_csr = csr_matrix(g_adj_mat)

In [34]:
%time spm = shortest_path(g_csr, directed=False)

CPU times: user 3min 29s, sys: 529 ms, total: 3min 29s
Wall time: 3min 29s


In [37]:
%time bp  = simple_bin_matrix(g_adj_mat)

CPU times: user 1.9 s, sys: 27.9 ms, total: 1.93 s
Wall time: 1.93 s


In [38]:
bp.shape

(20000, 609)

## Next Steps

* consider plotting to see which bins have the most items.
* consider writing a function that
  - picks some random nodes
  - find the bins associated with those nodes
  - comes up with selection list of bins and counts
 * migrate a selectio matrix in place. 
* bonus: try cugraph for shortest path calculation (probably not worth it).



In [ ]:
def gen_fake_sample_request(per_sample=20):
    pass

def gen_sample_like(nodeList, binMatrix):
    '''generate a sample by selecting random nodes that are mapped to the same bins as nodes in `nodelist`
    
    Parameters
    ----------
    nodeList (ndarray): 1d list of node indices of nodes to match against
    binList (ndarray): 2d matrix of  of node-bin-mapping. Each row is one-hot encoded matrix selecting which bin the node lives in.
    
    
    Output
    ------
    out - 1d array - sample that has nodes of the same degree as the sample represented by nodelist
    
    '''

    whichBins = gen_sample_request(nodeList, binMatrix)
    #TODO: left off here. this function was written in the foodome NMP Example Matrix notebook. 
    #the binIds and counts_per_bin seem kind of weird. I can still make this work. convert to more dense form or just use arange for binIds and change the function impl to handle zeros.
    
    gen_samples_from_multiple_bins2(binMatrix, binIds, counts_per_bin, 1):
    
def gen_sample_request(nodelist, binMatrix):
    '''Generate an array (one cell for each bin) that tells you how many nodes from each bin you will select to create your sample. 
    
    Parameters
    ----------
    
    Notes: you could implement this with onehot list of bins for each node then sum down the colum axis. 
    
    '''
    
    selectedBins = binMatrix[nodeList, :]
    return np.sum(selectedBins, axis=-1)


In [42]:
a = np.array([[0,1,0,0],
              [0,1,0,0],
              [0,0,0,1]])
ad = np.array([1,1,3])



In [43]:
a[[2,1],:]

array([[0, 0, 0, 1],
       [0, 1, 0, 0]])